## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various binary classifiers

In [ ]:
# Exploratory data analysis and visualization

In [1]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42
k_fold_splits = 2

np.random.seed(global_random_state)


active_pct = 0.073125471
inactive_pct = 1 - active_pct

# We set the inactive to have the weight of the active, and vice versa, to account for imbalance
class_weights = { 0: active_pct, 1: inactive_pct }

In [7]:
# Load assay info. Note: This CSV was obtained from PubChem bioassay aka PCBA, via searching for AID 1030 
# and downloading the datatable

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

# Delete CID 3246048, which fails featurization

compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

#fingerprint_df = 

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is None:
        print("Molecule failed featurization")
        print(index)
    else: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)
    
    if index % 10000 == 0:
        print("Processed index: {0}".format(index))

fingerprints = np.array(fingerprints)
activities = np.array(activities,dtype=int)
        
print("Processed all, saving non-sampled")

# Pickle the data to save time in the future
with open('data.classification.nonsampled.pickle', 'wb') as f:
    pickle.dump((fingerprints,activities), f, pickle.HIGHEST_PROTOCOL)
    
print("Sampling")

rus = RandomUnderSampler(random_state=global_random_state)
X, y = rus.fit_sample(fingerprints, activities)

print("Processed all, pickling")

#compound_ids_and_features = (compound_ids, smiles_list, fingerprints, activities)

# Pickle the data to save time in the future
with open('data.classification.undersampled.pickle', 'wb') as f:
    pickle.dump((X,y), f, pickle.HIGHEST_PROTOCOL)

Processed index: 0
Processed index: 10000
Processed index: 20000
Processed index: 30000
Processed index: 40000
Processed index: 50000
Processed index: 60000
Processed index: 70000
Processed index: 80000
Processed index: 90000
Processed index: 100000
Processed index: 110000
Processed index: 120000
Processed index: 130000
Processed index: 140000
Processed index: 150000
Processed index: 160000
Processed index: 170000
Processed index: 180000
Processed index: 190000
Processed index: 200000
Processed index: 210000
Molecule failed featurization
218052
Processed index: 220000
Processed all, sampling
Processed all, pickling


In [2]:
# Here we see that a naive approach (without balancing data) yields a poor F1-score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_predict
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from collections import Counter

global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)


In [3]:
skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True, random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = DecisionTreeClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    print("Computed roc_auc score of: {}".format(auc))
    print(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score is: {}".format(roc_auc_avg))


# Note: Unfortunately it's not directly comparable to ROC_AUC calculated in MoleculeNet at: https://arxiv.org/pdf/1703.00564.pdf 
# This is because MoleculeNet looks at a different metric (roc_auc) and also a different task (multiclass prediction across 128 bioassays simultaneously vs binary classification here)

Computed roc_auc score of: 0.6342477656405163
             precision    recall  f1-score   support

          0       0.63      0.63      0.63      8056
          1       0.63      0.64      0.63      8056

avg / total       0.63      0.63      0.63     16112

Computed roc_auc score of: 0.6350093109869646
             precision    recall  f1-score   support

          0       0.64      0.62      0.63      8055
          1       0.63      0.65      0.64      8055

avg / total       0.64      0.64      0.63     16110

Average roc_auc score is: 0.6346285383137404


In [4]:
# Does an MLP classifier help?
# Answer: Nope, not really

from sklearn.neural_network import MLPClassifier

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = MLPClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    print("Computed roc_auc score of: {}".format(auc))
    print(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score is: {}".format(roc_auc_avg))



Computed roc_auc score of: 0.6899205561072493
             precision    recall  f1-score   support

          0       0.68      0.71      0.70      8056
          1       0.70      0.67      0.68      8056

avg / total       0.69      0.69      0.69     16112

Computed roc_auc score of: 0.6955307262569832
             precision    recall  f1-score   support

          0       0.70      0.68      0.69      8055
          1       0.69      0.71      0.70      8055

avg / total       0.70      0.70      0.70     16110

Average roc_auc score is: 0.6927256411821163


In [5]:
# Let's try using a Random forest

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = RandomForestClassifier(n_estimators=100, random_state=global_random_state, n_jobs=-1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    print("Computed roc_auc score of: {}".format(auc))
    print(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score is: {}".format(roc_auc_avg))

Computed roc_auc score of: 0.7156777557100298
             precision    recall  f1-score   support

          0       0.70      0.75      0.72      8056
          1       0.73      0.69      0.71      8056

avg / total       0.72      0.72      0.72     16112

Computed roc_auc score of: 0.7162631905648665
             precision    recall  f1-score   support

          0       0.71      0.73      0.72      8055
          1       0.72      0.71      0.71      8055

avg / total       0.72      0.72      0.72     16110

Average roc_auc score is: 0.7159704731374481


In [ ]:
# Let's try using a hyperparameter tuning package
from hpsklearn import HyperoptEstimator

estim = HyperoptEstimator(trial_timeout=300, classifier=any_classifier('clf'))

# Search the space of classifiers and preprocessing steps and their
# respective hyperparameters in sklearn to fit a model to the data
estim.fit( X_train, y_train )

# Make a prediction using the optimized model
test_label = estim.predict( x_test )

# Report the accuracy of the classifier on a given set of data
score = estim.score( x_test, test_label )

# Return instances of the classifier and preprocessing steps
model = estim.best_model()

estim.fit( X_train, y_train )

print(estim.score(X_test, y_test))
print(estim.best_model())